In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),          # Resize to match model input size
    transforms.RandomHorizontalFlip(),    # Flip horizontally for variety
    transforms.RandomRotation(15),        # Small rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust colors
    transforms.ToTensor(),                # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

dataset_path = './Dataset'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
image, label = train_data[0]

In [ ]:
image.size()

In [ ]:
class_names = dataset.classes

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 15, 5)
        self.bn1 = nn.BatchNorm2d(15)
        self.conv2 = nn.Conv2d(15, 30, 5)
        self.bn2 = nn.BatchNorm2d(30)
        self.conv3 = nn.Conv2d(30, 60, 3)  # New convolutional layer
        self.bn3 = nn.BatchNorm2d(60)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(60, 120)  # Adjusted for the new layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        
        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Apply new layer
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

In [ ]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

In [ ]:
new_transforms = transforms.Compose([
    transforms.Resize((32, 32)),          # Resize to match model input size
    transforms.RandomHorizontalFlip(),    # Flip horizontally for variety
    transforms.RandomRotation(15),        # Small rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust colors
    transforms.ToTensor(),                # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize  
])

In [ ]:
def load_image(image_path):
    image = Image.open(image_path)
    image = new_transforms(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['wolf.jpg', 'hyena.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image  in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')